# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [6]:
w.shape

(2435, 65)

In [7]:
b.shape

(2435, 65)

In [8]:
wcall = w['call'].sum()
bcall = b['call'].sum()

In [9]:
wcall

235.0

In [10]:
p1 = wcall/w.shape[0]
p1

0.096509240246406572

In [11]:
bcall

157.0

In [12]:
p2 = bcall/b.shape[0]
p2

0.064476386036960986

### 1. What test is appropriate for this problem? Does CLT apply?

The most appropriate test here is a two sample z-test. CLT is applicable, cause resumes were randomly assigned 'black' or 'white' names, and we have a large enough sample size here, and each sample size is less than 10% of the population size. Let $p_1$ be the proportion of white named resume callback, and $p_2$ be the proportion of black named resume callbacks.

### 2. What are the null and alternate hypotheses?

H0: $p_1$ - $p_2$ = 0

H1: $p_1$ - $p_2$ != 0

The null hypothesis(H0) is that the observed difference between the samples is due to random variation, and that the proportion of returned calls in the sample population is in fact equal, while the alternate(H1) is that the proportion of callbacks for resumes with white sounding names in the sample population is significantly larger than those with black sounding names. Alpha is $\alpha$ = .05

### 3.Compute margin of error, confidence interval, and p-value. 

In [13]:
#p-value
from statsmodels.stats import weightstats as stests 
z = stests.ztest(w['call'],b['call'])
print('z-statistic: %.5f, p-value: %.5f' % z)

z-statistic: 4.11471, p-value: 0.00004


In [14]:
#confidence interval
import statsmodels.stats.api as sms
cm = sms.CompareMeans(sms.DescrStatsW(w['call']),sms.DescrStatsW(b['call']))
print(cm.tconfint_diff(usevar='unequal'))

C:\Users\wibot\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


(0.016770673983991798, 0.047295034434899373)


In [15]:
#margin error
me = w['call'].mean()-b['call'].mean() - cm.tconfint_diff(usevar='unequal')[0]
me

0.01526218087106878

### 4. Write a story describing the statistical significance in the context or the original problem.

This data provides that 235 resumes with white sounding names and 157 with black sounding names received call backs. For a proportion of 0.0965 and 0.0645 callbacks to non-callbacks respectively. We explored the difference in the proportion of callbacks in each group to see if white resumes had a higher callback rate compared to black resumes. To do this, we conducted a two sample z test for the difference in proportions. Z-score we get was 4.11471, with a p-value of 0.00004. As our p-value is smaller than our alpha of $\alpha$ = .05, we reject the null hypothesis and there is statistically significant evidence for the alternate hypothesis, that white sounding names receive a greater proportion of callbacks than black sounding names.

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

As we can see from the above race is a significant factor in callback success, but this does not necessarily mean that it's the most important factor. We only know that black sounding names receive fewer call backs when all other factors are equal.  

In [16]:
corr = data.corr(method='pearson')
corr['call'].sort_values(ascending=True)

fracdropout          -0.056671
lmedhhinc_empzip     -0.049879
req                  -0.041699
educreq              -0.033864
orgreq               -0.033416
fracwhite_empzip     -0.032989
branch_sales         -0.029126
computerskills       -0.028813
manuf                -0.028785
workinschool         -0.027888
branch_emp           -0.026909
l                    -0.025835
compreq              -0.024907
fracblack            -0.022130
trade                -0.021853
salesrep             -0.021584
military             -0.020577
manager              -0.020269
expreq               -0.019250
supervisor           -0.012061
bankreal             -0.008996
col                  -0.008479
missind              -0.007555
education            -0.005748
ofjobs                0.002311
retailsales           0.002336
comreq                0.002421
eoe                   0.003092
secretary             0.004038
busservice            0.006882
linc_empzip           0.006883
volunteer             0.007197
parent_s

As we can see from the above there are some highly correlated with callback features, such as special skills, years of experience and some other.